# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

## Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
x= 30
y= 120
z= 120

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(1,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches):           # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size):        # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image, (120,120))
                    temp = temp/127.5 - 1 #Normalize data
                    
                    
                    batch_data[folder,idx,:,:,0] = temp[:,:,0]  #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = temp[:,:,1]  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = temp[:,:,2]  #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch:", num_batches+1, "Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size, x,y,z,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = temp[:,:,0]
                    batch_data[folder,idx,:,:,1] = temp[:,:,1]
                    batch_data[folder,idx,:,:,2] = temp[:,:,2]
                
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
#train_path = '/notebooks/storage/Final_data/Collated_training/train'
#val_path = '/notebooks/storage/Final_data/Collated_training/val'
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model_1 = Sequential()

model_1.add(Conv3D(8, #filter size
                  kernel_size=(3,3,3),strides=(1,1,1) , input_shape=(x,y,z,3),padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2,2,1)))

model_1.add(Conv3D(16, #filter size
                  kernel_size=(3,3,3),strides=(1,1,1) ,padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(32, #filter size
                  kernel_size=(3,3,3),strides=(1,1,1) ,padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(64, #filter size
                  kernel_size=(3,3,3),strides=(1,1,1) ,padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_1.add(Flatten())

model_1.add(Dense(512, activation='relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)   #write your optimizer
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 120, 8)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 120, 16)   3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 120, 16)   64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 120, 16)   0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=0, verbose=1)  # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [37]:
model_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
Source path =  ./Project_data/train ; batch size = 32
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/21 [>.............................] - ETA: 4:11 - loss: 2.7503 - categorical_accuracy: 0.1875Batch: 4 Index: 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 120s 6s/step - loss: 1.9834 - categorical_accuracy: 0.3183 - val_loss: 1.4923 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init_2020-08-1318_35_50.775917/model-00001-1.99078-0.31674-1.49233-0.35000.h5
Epoch 2/10
21/21 [==============================] - 52s 2s/step - loss: 1.3001 - categorical_accuracy: 0.4990 - val_loss: 1.1710 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-08-1318_35_50.775917/model-00002-1.30011-0.49896-1.17102-0.50000.h5
Epoch 3/10
21/21 [==============================] - 46s 2s/step - loss: 1.0167 - categorical_accuracy: 0.5531 - val_loss: 1.0244 - val_categorical_accuracy: 0.5625

Epoch 00003: saving model to model_init_2020-08-1318_35_50.775917/model-00003-1.02673-0.55035-1.02443-0.56250.h5
Epoch 4/10
21/21 [==============================] - 45s 2s/step - loss: 0.9929 - categorical_accuracy: 0.6090 - val_loss: 0.6177 - val_categorical_accuracy: 0.8125

Epoch 00004

### Model 2 with batch size as 20 and different generator

In [11]:
train_doc_1 = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc_1 = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size_1 = 20 #experiment with the batch size

#### New Generator function

In [12]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [13]:
curr_dt_time = datetime.datetime.now()
#train_path = '/notebooks/storage/Final_data/Collated_training/train'
#val_path = '/notebooks/storage/Final_data/Collated_training/val'
train_path_1 = './Project_data/train'
val_path_1 = './Project_data/val'
num_train_sequences = len(train_doc_1)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc_1)
print('# validation sequences =', num_val_sequences)
num_epochs_1 = 6 # choose the number of epochs
print ('# epochs =', num_epochs_1)

# training sequences = 663
# validation sequences = 100
# epochs = 6


In [14]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define model 2
model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(classes, activation='softmax'))

model_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_6 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

In [15]:
train_generator_2 = generator_ex(train_path_1, train_doc_1, batch_size_1)
val_generator_2 = generator_ex(val_path_1, val_doc_1, batch_size_1)

In [18]:
model_2.fit_generator(train_generator_2, steps_per_epoch=steps_per_epoch, epochs=num_epochs_1, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator_2, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/trainEpoch 1/6
 ; batch size = 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 134s 6s/step - loss: 1.9706 - categorical_accuracy: 0.2024 - val_loss: 1.5923 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-08-1509_51_23.159512/model-00001-1.97056-0.20238-1.59225-0.20000.h5
Epoch 2/6
11/21 [==============>...............] - ETA: 56s - loss: 1.5782 - categorical_accuracy: 0.2091 Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 79s 4s/step - loss: 1.5449 - categorical_accuracy: 0.2367 - val_loss: 1.5306 - val_categorical_accuracy: 0.3125

Epoch 00002: saving model to model_init_2020-08-1509_51_23.159512/model-00002-1.56504-0.22472-1.53064-0.31250.h5
Epoch 3/6
21/21 [==============================] - 17s 791ms/step - loss: 1.5191 - categorical_accuracy: 0.3016 - val_loss: 1.5545 - val_categorical_accuracy: 0.1875

Epoch 00003: saving model to model_init_2020-08-1509_51_23.159512/model-00003-1.51912-0.30159-1.55453-0.18750.h5
Epoch 4/6
21/21 [==============================] - 17s 792ms/step - loss: 1.5927 - categorical_accuracy: 0.2540 - val_loss: 1.5423 - val_categorical_accuracy: 0.3125

Epoch 00004: saving model to model_init_2020-08-1509_51_23.159512/model-00004-1.59273-0.25397-1.54227-0.31250.h5

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/6
21/21 [==============================] - 17s 792ms/step - loss: 1.5586 - categoric

#### Model 3 with first generator and batch size 10 and (Y,Z) = (60, 60)

In [16]:
## changing x, y,z values
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [17]:
## changing batch size to-10
batch_size_3 = 10

In [18]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.losses import categorical_crossentropy
from keras import optimizers
import keras

#write your model here
model_3 = Sequential()

model_3.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_3.add(Activation('relu'))
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(classes, activation='softmax'))

model_3.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [19]:
train_generator_3 = generator_ex(train_path_1, train_doc_1, batch_size_3)
val_generator_3 = generator_ex(val_path_1, val_doc_1, batch_size_3)

In [20]:
model_3.fit_generator(train_generator_3, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator_3, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 66s 3s/step - loss: 1.6570 - categorical_accuracy: 0.1714 - val_loss: 1.6093 - val_categorical_accuracy: 0.2250

Epoch 00001: saving model to model_init_2020-08-1613_27_33.070135/model-00001-1.65702-0.17143-1.60932-0.22500.h5
Epoch 2/10
21/21 [==============================] - 52s 2s/step - loss: 1.6105 - categorical_accuracy: 0.2095 - val_loss: 1.6100 - val_categorical_accuracy: 0.1750

Epoch 00002: saving model to model_init_2020-08-1613_27_33.070135/model-00002-1.61045-0.20952-1.61001-0.17500.h5
Epoch 3/10
21/21 [==============================] - 60s 3s/step - loss: 1.6094 - categorical_accuracy: 0.1905 - val_loss: 1.6100 - val_categorical_accuracy: 0.2750

Epoch 00003: saving model to model_init_2020-08-1613_27_33.070135/model-00003-1.60942-0.19048-1.61005-0.27500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
 2/21 [=>............................] - ETA: 41s - loss: 1.6109 - categorical_accura

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 13s 600ms/step - loss: 1.6111 - categorical_accuracy: 0.1459 - val_loss: 1.6087 - val_categorical_accuracy: 0.2000

Epoch 00004: saving model to model_init_2020-08-1613_27_33.070135/model-00004-1.61123-0.14286-1.60871-0.20000.h5
Epoch 5/10
21/21 [==============================] - 8s 396ms/step - loss: 1.6098 - categorical_accuracy: 0.1587 - val_loss: 1.6064 - val_categorical_accuracy: 0.3250

Epoch 00005: saving model to model_init_2020-08-1613_27_33.070135/model-00005-1.60977-0.15873-1.60636-0.32500.h5
Epoch 6/10
21/21 [==============================] - 8s 379ms/step - loss: 1.6069 - categorical_accuracy: 0.2540 - val_loss: 1.6117 - val_categorical_accuracy: 0.1750

Epoch 00006: saving model to model_init_2020-08-1613_27_33.070135/model-00006-1.60693-0.25397-1.61174-0.17500.h5
Epoch 7/10
21/21 [==============================] - 8s 373ms/step - loss: 1.6095 - categorical_accuracy: 0.2857 - val_loss: 1.6111 - val_categorical_accuracy: 0.1750

Epo

#### Model 3 with batch size 20 experiment

In [21]:
## changing batch size to-20
batch_size_4 = 20

In [22]:
train_generator_4 = generator_ex(train_path_1, train_doc_1, batch_size_4)
val_generator_4 = generator_ex(val_path_1, val_doc_1, batch_size_4)

In [23]:
model_3.fit_generator(train_generator_4, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator_4, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 51s 2s/step - loss: 1.5677 - categorical_accuracy: 0.2143 - val_loss: 1.5137 - val_categorical_accuracy: 0.2375

Epoch 00001: saving model to model_init_2020-08-1613_27_33.070135/model-00001-1.56767-0.21429-1.51370-0.23750.h5
Epoch 2/10
11/21 [==============>...............] - ETA: 22s - loss: 1.4760 - categorical_accuracy: 0.2636Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 30s 1s/step - loss: 1.5194 - categorical_accuracy: 0.2305 - val_loss: 1.4452 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2020-08-1613_27_33.070135/model-00002-1.48486-0.25468-1.44515-0.40000.h5
Epoch 3/10
21/21 [==============================] - 7s 355ms/step - loss: 1.4612 - categorical_accuracy: 0.4444 - val_loss: 1.4341 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2020-08-1613_27_33.070135/model-00003-1.46119-0.44444-1.43405-0.30000.h5
Epoch 4/10
21/21 [==============================] - 9s 419ms/step - loss: 1.5689 - categorical_accuracy: 0.2857 - val_loss: 1.4988 - val_categorical_accuracy: 0.4250

Epoch 00004: saving model to model_init_2020-08-1613_27_33.070135/model-00004-1.56894-0.28571-1.49879-0.42500.h5
Epoch 5/10
21/21 [==============================] - 9s 422ms/step - loss: 1.4398 - categorical_accuracy: 0.4127 - val_loss: 1.4117 - val_categorical_accuracy: 0.3375

Epoch 

#### Model 2 with batch size 20 and epoch -10 experiment gave accuracy of 17%.
#### So we will go with model_3 and batch size as 30- accuracy was 32% (not a good model)
#### We will increase the batch_size to 40 and use different optimizers Adam delta.

In [24]:
## changing batch size to-30
batch_size_5 = 40
#x = 30 # number of frames
#y = 120 # image width
#z = 120 # image height
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [25]:
model_3.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [26]:
train_generator_5 = generator_ex(train_path_1, train_doc_1, batch_size_5)
val_generator_5 = generator_ex(val_path_1, val_doc_1, batch_size_5)

In [27]:
model_3.fit_generator(train_generator_5, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator_5, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 40
Source path =  ./Project_data/train ; batch size = 40
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Batch:  3 Index: 40
 1/21 [>.............................] - ETA: 4:08 - loss: 1.2813 - categorical_accuracy: 0.4500

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 86s 4s/step - loss: 1.3210 - categorical_accuracy: 0.4077 - val_loss: 1.1855 - val_categorical_accuracy: 0.4917

Epoch 00001: saving model to model_init_2020-08-1613_27_33.070135/model-00001-1.32799-0.40662-1.18555-0.49167.h5
Epoch 2/20
21/21 [==============================] - 52s 2s/step - loss: 1.2499 - categorical_accuracy: 0.4658 - val_loss: 1.1053 - val_categorical_accuracy: 0.5750

Epoch 00002: saving model to model_init_2020-08-1613_27_33.070135/model-00002-1.24988-0.46584-1.10529-0.57500.h5
Epoch 3/20
21/21 [==============================] - 44s 2s/step - loss: 1.2394 - categorical_accuracy: 0.4389 - val_loss: 1.5048 - val_categorical_accuracy: 0.3625

Epoch 00003: saving model to model_init_2020-08-1613_27_33.070135/model-00003-1.24056-0.43796-1.50482-0.36250.h5
Epoch 4/20
21/21 [==============================] - 41s 2s/step - loss: 1.1939 - categorical_accuracy: 0.4812 - val_loss: 1.0235 - val_categorical_accuracy: 0.5250

Epoch 00004:

KeyboardInterrupt: 

#### At this point we have categorical_accuracy: 85% in training data
#### The batch_size_5 = 40 and Epoch we got 85% is at 14/20 

#### Final Model